In [ ]:
import os, sys

REPO_URL = "https://github.com/vongrossi/fazendo-um-llm-do-zero.git"
REPO_DIR = "fazendo-um-llm-do-zero"

if not os.path.exists(REPO_DIR):
    !git clone {REPO_URL}

os.chdir(REPO_DIR)
sys.path.append(os.getcwd())

print("Diretório atual:", os.getcwd())


# Capítulo 07 — Instruction Tuning

Neste notebook vamos ensinar o modelo a seguir instruções humanas.

Até agora o modelo aprendeu:

• Como linguagem funciona  
• Como gerar texto  
• Como resolver tarefas supervisionadas  

Agora vamos ensinar o modelo a:

👉 Interpretar instruções  
👉 Responder perguntas  
👉 Produzir respostas orientadas  

Este é o passo que transforma modelos base em assistentes conversacionais.


Imports e configuração de uso de GPU

In [ ]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import random
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

seed_everything(42)


Importando criado GPTMini

from lib.gptmini import GPTConfig, GPTMini


Carregar Pesos do criado no Capítulo 05

In [ ]:
checkpoint_path = "05-pre-treinamento/gpt_checkpoint.pt"

config = GPTConfig(
    vocab_size=2000,
    context_size=64,
    d_model=128,
    n_heads=4,
    n_layers=2
)

backbone = GPTMini(config).to(device)

try:
    ckpt = torch.load(checkpoint_path, map_location=device)
    backbone.load_state_dict(ckpt, strict=False)
    print("Pesos do Cap 05 carregados ✅")
except:
    print("Checkpoint não encontrado — usando modelo inicial")


### 📊 Parte A — Dataset de Instruções

Dataset Médio de Instruções

In [ ]:
dataset = [
    {
        "instruction": "Explique o que é Machine Learning",
        "response": "Machine Learning é uma área da inteligência artificial que permite que sistemas aprendam padrões a partir de dados."
    },
    {
        "instruction": "Traduza para inglês: bom dia",
        "response": "Good morning"
    },
    {
        "instruction": "Explique o que é um token em LLMs",
        "response": "Token é uma unidade de texto convertida em representação numérica usada pelo modelo."
    },
    {
        "instruction": "Resuma: Modelos GPT usam Transformers",
        "response": "Modelos GPT utilizam arquitetura Transformer para processar linguagem."
    }
]


Função de Prompt Estruturado

In [ ]:
def format_prompt(item):
    return f"""
### Instrução:
{item['instruction']}

### Resposta:
{item['response']}
""".strip()


Tokenização Simples (char-level)

In [ ]:
all_text = "".join([format_prompt(d) for d in dataset])

chars = sorted(set(all_text))
stoi = {c:i for i,c in enumerate(chars)}
itos = {i:c for c,i in stoi.items()}

vocab_size = len(chars)


Encoder

In [ ]:
def encode(text):
    return [stoi[c] for c in text if c in stoi]

def decode(tokens):
    return "".join([itos[t] for t in tokens])


Criar Dataset com Masking

In [ ]:
def build_instruction_dataset(data, context_size=64):

    X, Y, mask = [], [], []

    for item in data:
        prompt = f"### Instrução:\n{item['instruction']}\n\n### Resposta:\n"
        full = prompt + item["response"]

        prompt_tokens = encode(prompt)
        full_tokens = encode(full)

        for i in range(len(full_tokens)-context_size):

            x = full_tokens[i:i+context_size]
            y = full_tokens[i+1:i+context_size+1]

            m = [0]*len(prompt_tokens)
            m = m + [1]*(context_size-len(prompt_tokens))

            X.append(x)
            Y.append(y)
            mask.append(m[:context_size])

    return (
        torch.tensor(X),
        torch.tensor(Y),
        torch.tensor(mask)
    )

context_size = 64
X, Y, MASK = build_instruction_dataset(dataset, context_size)

X, Y, MASK = X.to(device), Y.to(device), MASK.to(device)


### 📊 Parte B — Modelo Instruction Tuned
Wrapper com Loss Mascarada

In [ ]:
class InstructionGPT(nn.Module):

    def __init__(self, backbone):
        super().__init__()
        self.backbone = backbone

    def forward(self, x, y=None, mask=None):

        logits, _ = self.backbone(x)

        loss = None

        if y is not None:

            loss = F.cross_entropy(
                logits.view(-1, logits.size(-1)),
                y.view(-1),
                reduction="none"
            )

            loss = loss * mask.view(-1)
            loss = loss.mean()

        return logits, loss

model = InstructionGPT(backbone).to(device)


### 📊 Parte C — Treinamento SFT
Otimizador

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)


Loop de Treino

In [ ]:
loss_history = []

for step in range(300):

    idx = torch.randint(0, X.size(0), (8,))
    xb, yb, mb = X[idx], Y[idx], MASK[idx]

    logits, loss = model(xb, yb, mb)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    loss_history.append(loss.item())

    if step % 50 == 0:
        print(step, loss.item())


Visualizar Loss

In [ ]:
import matplotlib.pyplot as plt

plt.plot(loss_history)
plt.title("Loss Instruction Tuning")
plt.show()


### 📊 Parte D — Comparação Base vs Instruction Tuned
Função de Geração

In [ ]:
@torch.no_grad()
def generate(model, start, max_tokens=80):

    tokens = encode(start)
    tokens = torch.tensor(tokens).unsqueeze(0).to(device)

    for _ in range(max_tokens):
        logits, _ = model(tokens)
        next_token = torch.argmax(logits[:, -1, :], dim=-1)
        tokens = torch.cat([tokens, next_token.unsqueeze(1)], dim=1)

    return decode(tokens.squeeze().tolist())


Testando o modelo !

In [ ]:
prompt = "### Instrução:\nExplique o que é Machine Learning\n\n### Resposta:\n"

print(generate(model, prompt))


### 📊 Parte E — Salvar Checkpoint Final


In [ ]:
torch.save(model.state_dict(), "07_instruction_gpt.pt")


# Encerramento da Série

Neste capítulo você ensinou um modelo a:

• Interpretar instruções  
• Produzir respostas orientadas  
• Aprender comportamento conversacional  

Você percorreu toda a jornada:

Texto → Tokens → Atenção → GPT → Treinamento → Fine-Tuning → Instruction Tuning

Este é o pipeline fundamental dos assistentes baseados em LLMs modernos.
